# Классы и интерфейс sklearn

### Что такое классы

Грубо говоря, класс - это набор функций, который может также хранить какие-то внутренние переменные. Применять класс удобно в случаях, когда требуется вызывать последовательность функций. Тогда вместо того, чтобы передавать переменные - результат работы одной функции в другую функцию, эти переменные хранят внутри класса.

Создадим класс, который ищет максимум или минимум из чисел, которые будут подавать через функцию add:

In [84]:
class Optim:
    def __init__(self, find_max=True):
        if find_max:
            self.opt = 10**(-10)
        else:
            self.opt = 10**(10)
        print("I am initializing")
        self.find_max = find_max
        
    def add(self, x):
        if self.find_max:
            if x > self.opt:
                self.opt = x
        else:
            if x < self.opt:
                self.opt = x
                
    def compute(self):
        print("Optimumm equals ", self.opt)

Чтобы воспользоваться классом, нужно сначала создать переменную этого класса (или объект класса):

In [94]:
optim = Optim()

I am initializing


Внутри класса хранится переменная opt:

In [95]:
optim.opt

1e-10

Теперь можно вызывать функции класса, которые будут обновлять внутренние переменные:

In [96]:
optim.add(x=7)

In [97]:
optim.add(2)

In [98]:
optim.compute()

Optimumm equals  7


Можно создать другую переменную класса - она не будет никак связана с первой, в частности, у нее будет другое значение opt:

In [99]:
optim2 = Optim(find_max=False)

I am initializing


In [100]:
optim2.opt

10000000000

При создании объекта класса можно передавать параметры: эти параметры будут немного менять логику работы класса. Например, в нашем случае параметр find_max сообщает классу, нужно искать минимум или максимум. Создавать два отдельных класса под минимум и максимум не имеет смысла, потому что код почти одинаковый, но как-то сообщить классу, что именно он делает, нужно - вот и придумали параметры класса.

Теперь поговорим вкратце о синтаксисе создания класса:
* определение класса начинается строкой class Name:
* далее следуют функции класса. у каждой функции первый аргумент всегда называют self - это ссылка на самого себя, чтобы можно было обращаться к внутренним переменным (в нашем примере self.opt). После self следуют любые необходимые пользователю аргументы (в нашем случае - аргумент x функции add). При вызове функции python сам подставит ссылку на объект класса в self, так что пользователю нужно передавать только "свои" аргументы (в примерах мы пишем optim.add(2), а self перед 2 не пишем)
* выделяется одна служебная функция, именуемая \__init\__. Она вызывается при создании объекта класса (когда мы пишем Optim()). В этот момент прозводится инициализация внутренних переменных класса, запоминание параметров и любые другие операции, если они нужны пользователю. В нашем примере необходимо инициализировать переменную self.opt, а также запомнить, будем мы искать минимум или максимум. Также добавлен print, чтобы в примерах было видно, в какой момент вызывается \__init\__ 

### Интерфейс Scikit-Learn

Для реализации алгоритмов машинного обучения в sklearn всегда используется один интерфейс - класс с функциями fit(X, y) для обучения модели и predict(X) для возвращения предсказаний. При создании класса можно указывать дополнительные параметры, влияющие на работу алгоритма машинного обучения.

Пример с линейной регрессией:
* При создании класса нужно запомнить коэффициент регуляризации
* Задача функции fit - по выборке X и y найти веса w и сохранить их внутри класса в self.w
* Задача функции predict - по весам self.w и X вернуть y:

In [102]:
class LinearRegressor:
    def __init__(self, reg_coef=None):
        self.lambda_ = reg_coef
    
    def fit(self, X_train, y_train):
        self.w = 0 
        # тут вместо заглушки 0 надо написать формулу для вычисления весов по X, y и self.lambda_
    
    def predict(self, X_test):
        y_pred = X_test.dot(self.w)
        return y_pred

Если бы не использовали класс, нам пришлось бы передавать веса w в функцию predict каждый раз, когда мы захотели бы сделать предсказания, это неудобно (особенно если внутренних переменных много). А так веса хранятся внутри класса, и мы можем даже не догадываться об их существовании (если класс писали не мы)

Пример импорта классификатора из sklearn:

In [32]:
from sklearn.dummy import DummyClassifier

### Задача - реализовать класс для нормализации данных

Предобработка данных в sklearn реализована по похожему шаблону: функция fit(X) запоминает внутренние переменные, а функция transform(X) выполняет преобразование выборки. y здесь не нужен, потому что в нормализации целевые переменные не участвуют (как и почти во всей предобработке данных).

Реализуем класс для нормализации данных. Параметров у класса нет, так что функцию \__init\__ мы пропускаем. Функция fit считает статистики (по обучающей выборке), а функция tranform вычитает среднее и делит на стандартное отклонение. 

In [40]:
class Normalizer:
    def fit(self, X):
        self.mu = X.mean(axis=0)
        self.sigma = X.std(axis=0)
        
    def transform(self, X):
        return (X - self.mu[np.newaxis, :]) / self.sigma[np.newaxis, :]

Создаем случайные данные X и y для тестирования нашего класса:

In [41]:
import numpy as np

In [52]:
X_tr = np.random.randint(-5, 5, size=(20, 4))
X_tr.shape

(20, 4)

In [54]:
X_te = np.random.randint(-5, 5, size=(10, 4))
X_te.shape

(10, 4)

In [57]:
X_tr

array([[-5,  3, -2,  1],
       [ 1,  1, -3, -3],
       [ 0, -2,  4,  4],
       [ 3, -1, -5,  3],
       [ 1, -2,  0,  3],
       [-4, -1,  3, -5],
       [ 0,  2,  3, -1],
       [-3, -2, -2,  1],
       [-1, -1,  2, -4],
       [ 4,  2,  2,  4],
       [ 2,  3,  1,  1],
       [-1,  1, -1, -4],
       [ 3, -4, -5,  2],
       [-3,  2, -5, -2],
       [ 0,  1,  0,  3],
       [-5, -5, -3, -4],
       [-4, -2, -1,  2],
       [ 0, -4, -1, -4],
       [ 1,  4,  4, -4],
       [ 2,  4, -5, -1]])

Создаем объект класса и трансформируем выборку:

In [59]:
normalizer = Normalizer()
normalizer.fit(X_tr)
X_tr_transformed = normalizer.transform(X_tr)
X_te_transformed = normalizer.transform(X_te)

Fit нужно вызывать именно на обучающих данных, чтобы ничего не подсмотреть в контрольной выборке. А transform можно вызывать много раз для любых выборок (с уже посчитанным статистиками, которые хранятся внутри класса).

In [56]:
X_tr_transformed

array([[-1.70190108,  1.14890018, -0.43551639,  0.45809399],
       [ 0.54236408,  0.39552301, -0.77052899, -0.85074599],
       [ 0.16831989, -0.73454274,  1.57455924,  1.43972398],
       [ 1.29045247, -0.35785415, -1.4405542 ,  1.11251398],
       [ 0.54236408, -0.73454274,  0.23450882,  1.11251398],
       [-1.32785689, -0.35785415,  1.23954664, -1.50516598],
       [ 0.16831989,  0.7722116 ,  1.23954664, -0.196326  ],
       [-0.95381269, -0.73454274, -0.43551639,  0.45809399],
       [-0.20572431, -0.35785415,  0.90453403, -1.17795598],
       [ 1.66449666,  0.7722116 ,  0.90453403,  1.43972398],
       [ 0.91640828,  1.14890018,  0.56952143,  0.45809399],
       [-0.20572431,  0.39552301, -0.10050378, -1.17795598],
       [ 1.29045247, -1.4879199 , -1.4405542 ,  0.78530399],
       [-0.95381269,  0.7722116 , -1.4405542 , -0.52353599],
       [ 0.16831989,  0.39552301,  0.23450882,  1.11251398],
       [-1.70190108, -1.86460849, -0.77052899, -1.17795598],
       [-1.32785689, -0.